<a href="https://colab.research.google.com/github/aTegart/abc-music-gen/blob/main/D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Getting set up:

You will be connecting to your personal google drive, so make sure you have '/content/drive/My Drive/CS 486 Music Generation/' present (without the ending space, so Celine I changed your folder name). This code now reads from all song folders. Please make sure you are on the latest version of the songs, because on Nov 11 changes were made to filetypes and to remove accented characters. When uploading files to your personal google drive, make sure to have "convert uploads" unchecked in your settings.

Remember you need to authenticate when you remount, and copy the auth code from the link.





In [1]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


Check position


In [2]:
!pwd
!cd './drive/My Drive/'
!pwd

/content
/content


## Preprocess the songs

In [3]:
from os import listdir
from os.path import isfile, join
import os

#list songs
mypath = '/content/drive/My Drive/CS 486 Music Generation/' #this was previously "myPath" when it needed to be "mypath" 🤯
#/content/drive/My Drive/CS 486 Music Generation 
#/content/drive/My Drive/CS 486 Music Generation
folders = [f for f in listdir(mypath) if not isfile(join(mypath, f))]
print(folders)
allSongs = []
for folder in folders:
  #if folder == "montreal":
  for song in listdir(mypath + '/' + folder):
    print(folder+" "+song)
    f = open(os.path.join(mypath,folder,song), "r")
    text = f.read()
    allSongs.append(text)
print(len(allSongs))


['montreal', 'canad', 'tune arch canada']
montreal Aimee Gagnon - K G.txt
montreal August - K F.txt
montreal Berceuse pour Hannah - K D.txt
montreal Bonjour, comment ca va? - K G.txt
montreal Brandy - K G.txt
montreal Breakfast at the Methes - K Ddor.txt
montreal Danse de campagne - K D.txt
montreal Gigue Andre Alain - K D.txt
montreal Gigue back a back - K D.txt
montreal Gigue des capuchons - K D.txt
montreal Gigue du Demon - K D.txt
montreal Gigue du Diamant Bleu - K D.txt
montreal Hommage a Dorothee - K D.txt
montreal La belle Catherine - K D.txt
montreal La chaise noire - K Gmin.txt
montreal La Complainte du Folkloriste - K D.txt
montreal Le bonhomme et la bonne femme - K D .txt
montreal Le brandy - K D.txt
montreal Le Chat Noir - K Gmin.txt
montreal Valse du coq - K D.txt
montreal Reel des cinq jumelles - K D.txt
montreal Reel du Bonhomme - K D.txt
montreal Reel d'Alison - K D.txt
montreal Reel Chicoutimi - K A.txt
montreal Reel Beaulieu - K Amix.txt
montreal Reel a Cedulie - K F.

In [4]:
print(allSongs[100])

X:63
T:The Lucky Trapper
C:Andy Dejarlis
S:Doug Creighton
Z:robin.beech@mcgill.ca
R:reel
M:4/4
L:1/8
R:reel
K:G
D2 | G2BG DGBG | DGBG cGBG | A2AF DFAF | DFAc BGAF |
G2BG DGBG | cGBG ABAG | FGAB cdef | gfga g2 ::
d2 | g2ga gfed | eaab a2ag | f2d2 dedc | BGAF GABd |
g2ga gfed | eaab a2ag | fgaf bfaf | gfga g2 :|


In [5]:
#Combine songs into one long string
spacing = "\n\n"
combined_songs = spacing.join(allSongs)

#Find vocabulary for vectorization (unique characters in string)
uniq_chars = sorted(set(combined_songs))
print(len(uniq_chars))

char_num_map = {c:i for i, c in enumerate(uniq_chars)}
num_char_map = {i:c for i, c in enumerate(uniq_chars)}

#proof that the mapping holds both ways
print('{')
for char,_ in zip(char_num_map, range(20)):
# for char,_ in zip(char_num_map, range(len(char_num_map))):
    print('  {:4s}: {:3d},'.format(repr(char), char_num_map[char]))
print('  ...\n}')

print('{')
for char,_ in zip(num_char_map, range(20)):
# for char,_ in zip(num_char_map, range(len(num_char_map))):
    print('  {:4s}: {:3s},'.format(repr(char), num_char_map[char]))
print('  ...\n}')

112
{
  '\t':   0,
  '\n':   1,
  '\x1a':   2,
  ' ' :   3,
  '!' :   4,
  '"' :   5,
  '#' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  '+' :  13,
  ',' :  14,
  '-' :  15,
  '.' :  16,
  '/' :  17,
  '0' :  18,
  '1' :  19,
  ...
}
{
  0   : 	  ,
  1   : 
  ,
  2   :   ,
  3   :    ,
  4   : !  ,
  5   : "  ,
  6   : #  ,
  7   : %  ,
  8   : &  ,
  9   : '  ,
  10  : (  ,
  11  : )  ,
  12  : *  ,
  13  : +  ,
  14  : ,  ,
  15  : -  ,
  16  : .  ,
  17  : /  ,
  18  : 0  ,
  19  : 1  ,
  ...
}


In [30]:
import numpy as np
# Vectorize combined_songs
# array of a num for each char in the combined_songs string
vsongs = np.array([char_num_map[c] for c in combined_songs])

print ('{} ---- characters mapped to int ----> {}'.format(repr(combined_songs[:10]), vsongs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vsongs, np.ndarray), "returned result should be a numpy array"

'X:21\nT:Aim' ---- characters mapped to int ----> [58 28 20 19  1 54 28 35 74 78]


In [31]:
def batch(vsongs, seqlen, bsize):
  highest_start_index = vsongs.shape[0] - 1 - seqlen
  start_indices = np.random.choice(highest_start_index, bsize)

  input = [vsongs[i : i+seqlen] for i in start_indices]
  output = [vsongs[i+1 : i+1+seqlen] for i in start_indices]

  input = np.reshape(input, [bsize, seqlen])
  output = np.reshape(output, [bsize, seqlen])

  return input, output

testin, testout = batch(vsongs, 5, 2)
#observe how testout is shifted one num forward from testin
print(testin)
print(testout)

[[36  3 93  3 40]
 [ 3 93  3 66 22]]
[[ 3 93  3 40 36]
 [93  3 66 22 15]]


In [32]:
#more testing, remove later

x_batch, y_batch = batch(vsongs, seqlen=5, bsize=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(num_char_map[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(num_char_map[target_idx])))

Step   0
  input: 93 ('|')
  expected output: 3 (' ')
Step   1
  input: 3 (' ')
  expected output: 5 ('"')
Step   2
  input: 5 ('"')
  expected output: 35 ('A')
Step   3
  input: 35 ('A')
  expected output: 5 ('"')
Step   4
  input: 5 ('"')
  expected output: 70 ('e')


## Setup Tensorflow and our ML env

In [33]:
%tensorflow_version 2.x
import tensorflow as tf 
from tensorflow import keras

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0

In [34]:
## Set up the RNN model
def LSTM(rnn_units): 
  return keras.layers.LSTM(
    rnn_units, 
    return_sequences=True, 
    recurrent_initializer='glorot_uniform',
    recurrent_activation='sigmoid',
    stateful=True,
  )

## Build the RNN Model
def build_RNN_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = keras.Sequential([
    # 1: Embedding layer to transform indices into dense vectors of a fixed embedding size
    keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    # 2: LSTM with rnn_units number of units
    LSTM(rnn_units),
    # 3: Dense layer that transform the LSTM output into the vocab_size
    keras.layers.Dense(vocab_size)
  ])
  return model

# Build model with default hyperparameters which can be modified
model = build_RNN_model(len(uniq_chars), embedding_dim=256, rnn_units=1024, batch_size=32)

### Test the RNN Model

In [35]:
## Prints out a summary of the model so we can check for its layers, shapes, batch sizes, etc.
model.summary()

## Check output dimemsionality
x, y = batch(vsongs, seqlen=50, bsize=32)
pred = model(x)
print("Input shape: ", x.shape)
print("Prediction shape: ", pred.shape)

## Check the prediction of the untrained RNN model using categorical distribution
random_indices = tf.random.categorical(pred[0], num_samples=1)
random_indices = tf.squeeze(random_indices, axis=-1).numpy()
random_indices

## decode the prediction from the unntrained RNN model
## We should expect a prediction that does not make much sense as it is untrained
index2char = np.array(uniq_chars)
print("Input: \n", repr("".join(index2char[x[0]])))
print("Predicted next char: \n", repr("".join(index2char[random_indices])))

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (32, None, 256)           28672     
_________________________________________________________________
lstm_7 (LSTM)                (32, None, 1024)          5246976   
_________________________________________________________________
dense_7 (Dense)              (32, None, 112)           114800    
Total params: 5,390,448
Trainable params: 5,390,448
Non-trainable params: 0
_________________________________________________________________
Input shape:  (32, 50)
Prediction shape:  (32, 50, 112)
Input: 
 'd-freebsd\nLast-Modified: Sun, 27 Apr 2014 23:56:38'
Predicted next char: 
 'B“\\E57465)eçfO>lq#çJL/\n,!;qrA3@pWk&4“mâ,–-N\\V[Z\ty('


### Training the RNN model

In [36]:
## Train with sparse_categorical_corssentropy loss which uses integer targets for categorical classificiation tasks
## logits: predictions

def get_loss(labels, logits):
  return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

sample_loss = get_loss(y, pred) # from the sample prediction in the above section
print("Scalar loss: ", sample_loss.numpy().mean())


Scalar loss:  4.719783


### Hyperparameters

Write down the values you used and what you got for average. 
Pick high, mid and low values in the range given by the MIT lab.
- batch_size (2: 14% (5-28), 10: 39% (33-49), 50: 25% (12-37) --- overall diff. 39-14 = 25)
- training_cycles (1000: 29% (21-37), 2000: 39% (28-49), 3000: 40% (26-56) --- overall diff. 40-29 = 11)
- rnn_units(128: 31%(23-44), 256: 36%(23-49), 512: 41%(28-56), 800: 38%(21-49), 1024: 10%(2-23), overall diff 41-10=31)
- learning_rate(1e-5: 1%(0-5), 2e-3: 46%(30-48), 3e-3: 42%(33-49), 4e-3: 38%(30-47), 5e-3: 42%(28-51), 3e-2(24%(14-35), overall diff 46-24=22)
- seq_len(50: 33%(19-47), 100: 37%(28-49), 200: 27%(14-35), 250: 40%(26-49), 300: 45%(33-53)500: 39%(28-51), overall diff 45-27=18)


Default start values:

Optimization parameters:
- num_training_iterations = 2000  # Increase this to train longer
- batch_size = 4  # Experiment between 1 and 64
- seq_length = 100  # Experiment between 50 and 500
- learning_rate = 5e-3  # Experiment between 1e-5 and 1e-1

Model parameters: 
- vocab_size = len(vocab)
- embedding_dim = 256 
- rnn_units = 1024  # Experiment between 1 and 2048

In [138]:
### Define hyperarameters
# Model params
rnn_units = 512 # suggeted range: 1 - 2048, default: 1024
uniq_chars_len = len(uniq_chars)
embedding_dim = 256

# Optimization params
training_cycles = 3000 # increase to make longer trainings default: 2000
batch_size = 10 # suggested range: 1 - 64 default 4
seq_len = 300 # suggested range: 50 - 500 default 100
learning_rate = 2e-3 # suggested range: 1e-5 - 1e-1, default: 5e-3

# Checkpoint location
ckpt_dir = './training_checkpoints'
ckpt_prefix = os.path.join(ckpt_dir, "ckpt")

# A flag to indicate if we want to use the whole song
ISOBJTESTING = False

# Objective Test
Run this cell for objective testing only.

In [139]:
# divide strings into training and validation sets
def split_training_validation(all_songs):
  training_set = []
  validation_set = []
  for i in range(0, len(all_songs)):
    if i % 10 == 0:
      validation_set.append(all_songs[i])
    else:
      training_set.append(all_songs[i])
  return training_set, validation_set

# combine songs into one long string 
training_set, validation_set = split_training_validation(allSongs)
print(len(training_set))
print(len(validation_set))
training_combined = spacing.join(training_set)
# vectorize combined_songs
v_training = np.array([char_num_map[c] for c in training_combined])

ISOBJTESTING = True



380
43


### Train the RNN model

In [140]:
from tqdm import tqdm # progress bar

rnn_model = build_RNN_model(uniq_chars_len, embedding_dim, rnn_units, batch_size)
optimizer = keras.optimizers.Adam(learning_rate)

@tf.function
def step(x, y):
  with tf.GradientTape() as tape:
    y_hat = rnn_model(x)
    loss = get_loss(y, y_hat)
    gradients = tape.gradient(loss, rnn_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))
    return loss

# clear if there is progress bar instance
if hasattr(tqdm, '_instances') : tqdm._instances.clear()

## Start training
history = []

for i in tqdm(range(training_cycles)):
  x_batch = []
  y_batch = []
  # using the whole data set for training
  if not ISOBJTESTING:
    x_batch, y_batch = batch(vsongs, seq_len, batch_size)
  # using 90% of the data set
  else:
    x_batch, y_batch = batch(v_training, seq_len, batch_size)
  loss = step(x_batch, y_batch)
  history.append(loss.numpy().mean())
  # Update model with updated weights
  if i%seq_len == 0:
    rnn_model.save_weights(ckpt_prefix)

# save model
rnn_model.save_weights(ckpt_prefix)
# print(rnn_model.layers[0].get_weights()[0])

100%|██████████| 3000/3000 [01:58<00:00, 25.26it/s]


# Restore the lastest checkpoint



In [141]:
rnn_model = build_RNN_model(uniq_chars_len, embedding_dim, rnn_units, batch_size=1)
rnn_model.load_weights(tf.train.latest_checkpoint(ckpt_dir))
# print(rnn_model.layers[0].get_weights()[0])
rnn_model.build(tf.TensorShape([1, None]))
rnn_model.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_43 (Embedding)     (1, None, 256)            28672     
_________________________________________________________________
lstm_43 (LSTM)               (1, None, 512)            1574912   
_________________________________________________________________
dense_43 (Dense)             (1, None, 112)            57456     
Total params: 1,661,040
Trainable params: 1,661,040
Non-trainable params: 0
_________________________________________________________________


# Generate result

In [142]:
def generate_text(model, start_string, len=1000):
  input_vector = [char_num_map[s] for s in start_string]
  input_vector = tf.expand_dims(input_vector, 0)
  text_generated = []
  model.reset_states()
  tqdm._instances.clear()

  for i in range(len):
    predictions = model(input_vector)
    predictions = tf.squeeze(predictions, 0)
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
    input_vector = tf.expand_dims([predicted_id], 0)
    text_generated.append(num_char_map[predicted_id])
  
  return (start_string + ''.join(text_generated))

# Prediction
Run this cell to calculate accuracy.

In [143]:
def find_first_n_in_tune(song, n):
  key_index = song.find('K:')
  tune_index = song.find('\n', key_index) + 1
  # uncomment this if you only want the seed to include characters in the song
  # seed = song[tune_index: tune_index + n]
  # using everything up to the nth character of the song itself
  seed = song[:tune_index + n]
  return tune_index, seed

def find_accuracy(songs, n):
  correct = 0
  total = 0
  for song in songs:
    tune_index, seed = find_first_n_in_tune(song, n)
    if tune_index + n > len(song):
      print("INDEX OUT OF RANGE, SKIPPING\n")
      continue
    char_generated = generate_text(rnn_model, start_string=seed, len = 1)[-1]
    char_expected = song[tune_index + n]
    if char_generated == char_expected:
      correct += 1
    total += 1
   # print(song)
   # print("seed: \"" + seed + "\" expected: \"" + char_expected + "\" prediction: \"" + char_generated + '\"\n')
  return correct/total

# generated_text = generate_text(rnn_model, start_string=':', len=1000)
if ISOBJTESTING:
# Choose the seed as the first 8 tokens of a tune from our validation dataset
  outputs = []
  sum = 0
  for i in range(50):
    accuracy = find_accuracy(validation_set, 8)
    # print("\nAccuracy = {0:.0%}".format(accuracy))
    outputs.append(accuracy)
    sum += accuracy

  average = sum / 50.0
  print("\nAverage = {0:.0%}".format(average))
  print("\nMin: {0:.0%}".format(min(outputs)))
  print("\nMax: {0:.0%}".format(max(outputs)))
  
  



  


Average = 50%

Min: 37%

Max: 60%


# Output

In [153]:
generated_text = generate_text(rnn_model, start_string='X', len=1000)
print('\n' + generated_text)


X:1
Q:1/46
K:G
D2B,D GBAG | FDDD A,DDG (3ABA|Gg3e dBGB|e2de dBGB|e2 ef egfe|dcde fdBd|cBcd c2 GA:|2Bcde f2dB||


X:1
T:Reel de la Pistrol, Andy DF |]

X:153
T:Louis Waltz
R:Waltz
C:Trad.
O:Canada
M:3/4
L:1/8
Q:1/4=120
F:http://trillian.mit.edu/~jc/music/abc/mirror/gud.com/French_CDEprilan.mit.ed Beaudry
N:Steph Oninoux (New Yandes Ow https://4
K:D
P:A
D2 |D2 DE FE| F2 FE A|DFA d2ef|gecA BABc|d2 fd ed B2||


X: 1
T:Valse a Calier
C:Richard Forest
S:Marche Picor
O: Dor Messern Blan Rephsine (1924)
D:http://www.collectionscanada.ca/obj/m2/f7/1359/8c3/4C2C9/4C5/8=A] z/8 [^C/8^g13/4z5/4] [=C/8^A,5/4f5/4A9/8]
[G1/8z/2] [f5/8z/8] [e11/4E5/4^G19/8] "E9/8 z/8 [F,11/8^FD9/8] z3/8 [E9/8g/2] [G5/4B17/8c13/8z] z/8 [e5/8z/8] [f5/4z/8] [e9/8^C9/8z5/8]
[G,/8G/8] [B,19/8z/2] [C9/8c9/8=e7/2=F45/8c29/8] z/8 [d9/8B,5/4z5/8] [^f5/8z/8]
[G,19/8d15/8z/2] [F5/2A13/4c13/4E19/8] z3/2 [d3/4A/8]
[G13/8G,9/8B9/8] [e3/8z/8] [A,5/2A13/8z5/4] A,/8 [g13/8B,G77/8c3/4] z/8 [e3/8z/4] [G,/8d5/4B5/4z9/8] [^f/8
[c9/8F95/8c